## Demo Notebook for Accessing PR100 Data on Azure

Launched on February 2, 2022, a two-year study entitled Puerto Rico Grid Resilience and Transitions to 100% Renewable Energy (PR100) will perform a comprehensive analysis of stakeholder-driven pathways to Puerto Rico’s clean energy future. For more information, please visit [https://www.energy.gov/gdo/puerto-rico-grid-resilience-and-transitions-100-renewable-energy-study-pr100].

To support the PR100 project, the Open Energy Data Initiative has made an assortment of data sets available for free public access. This notebook will demonstrate how to access the PR100 data located in Azure BLOB storage.

### Get Access Token

You do not need an Azure account to access public data. Instead, you can obtain a temporary access token via the Planetary Computer's API. This can be accomplished via either the requests or planetary_computer libraries. Both options are shown below.

In [ ]:
# get a token with requests
import requests

token = requests.get(
    'https://planetarycomputer.microsoft.com/api/sas/v1/token/nrel/oedi'
).json()['token']

In [ ]:
# get a token with planetary-computer
import planetary_computer

token = planetary_computer.sas.get_token('nrel', 'oedi').token


### Explore Container

Use the token to create a PyFileSystem object. You can explore the contents of the container using the get_file_info method. The PR100 data consists of geoparquet and geotiff files that are organized into directories:
- Boundaries
- Habitat
- Hazards
- Infrastructure
- Topography

In [ ]:
from pyarrow.fs import PyFileSystem, FSSpecHandler, FileSelector
from adlfs import AzureBlobFileSystem

# Create file system
fs = PyFileSystem(
    FSSpecHandler(
        AzureBlobFileSystem('nrel', credential=token)
    )
)

# View files in the 'Boundaries' directory
fs.get_file_info(FileSelector('/oedi/PR100/Boundaries/'))

### Load Vector Data

Let's load one of those files into a geodataframe and visualize it.

In [ ]:
import geopandas

df = geopandas.read_parquet('oedi/PR100/Boundaries/land_protected_areas.parquet', filesystem=fs)
df.explore()

### Load Raster Data

If we look in the Topography directory, we'll see some tif files. These are cloud optimized GeoTiffs.

In [ ]:
fs.get_file_info(FileSelector('/oedi/PR100/Topography/'))

We can load these files with the rasterio package.

In [ ]:
import rasterio
import rasterio.plot

with fs.open_input_file('oedi/PR100/Topography/elevation.tif') as file:
    raster = rasterio.open(file)
    print(raster.meta)
    rasterio.plot.show(raster, adjust=True)